# 4. Data validation & additional data collection from PubChem

## Purpose, contents, & conclusions

**Purpose:** Prior notebooks extracted chemical information from a few different sources, ultimately parsing & writing it to `aromas-flavors.db` and `aromas-flavors-categorized.csv`. However, little has been done to verify the quality of the data aside from dropping rows completely missing key fields. Because chemical names will be used to generate chemical structures, it is important to verify them before use.

In this notebook, data is read from `aromas-flavors-categorized.csv` (where each row = one molecule, with name, CAS number, and flavor and aroma descriptors). Using the PubChem API, chemical names are verified & converted to common names where appropriate. If a chemical's name cannot be found, it is retrieved by CAS number instead. As an added bonus, additional structural information is also retrieved from PubChem at the same time.

**Contents:** The notebook contains:
* Functions to manage the data validation & extension through the PubChem API, including:
    * `manage_query()`: accepts a dict describing one molecule & manages lookup in the API
    * `get_pubchem_id()`: queries PubChem for a unique (but meaningless) ID number by name or, as a fallback, CAS number
    * `get_pubchem_data()`: queries PubChem for structural data using a molecule's PubChem ID number
    * `clean_name()`: applied only when a name match cannot be found, it aggressively simplifies the name

**Conclusions:** Key outputs are:
* `aromas-flavors-validated.csv`: A *tab*-separated file in which each row represents one molecule. Fields are the molecule's `common_name`, its chemical structure as both a `SMILES_code` and an `InChI_code`, and descriptions of its `aroma` and `flavor`.

## Import the required libraries

In [1]:
import csv
import requests

## Read the CSV data

In [2]:
aromas_flavors = []
with open("processed-data/aromas-flavors-categorized.csv", "r") as file:
    reader = csv.DictReader(file, delimiter = "\t")
    for entry in reader:
        aromas_flavors.append(entry)

In [3]:
aromas_flavors[0]

{'name': '(+)-Menthofuran',
 'CAS_num': '17957-94-7',
 'aroma': 'fruity floral fatty fruity|citrusy',
 'flavor': ''}

## Build the validation functions

PubChem has a [convenient API](https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest) for retrieving chemical data. From a brief exploration of its documentation, it seems like the most convenient way to retrieve data from a (potentially) nonstandard chemical name will be to first fuzzy-match the nae to a unique (but chemically meaningless) PubChem ID number. The ID number can then be used to look up additional information.

In [4]:
from time import sleep

In [5]:
def get_pubchem_id(name: str) -> str:
    """
    Called:
        By the `validate` function.
    Purpose:
        Search by a molecule's name for its unique PubChem ID number. Find
        the closest match (which is the first listed in the search results),
        if one exists.
    Accepts:
        name: the molecule's name as a string
    Returns:
        the molecule's unique PubChem ID number, which has no inherent
        chemical meaning whatsoever, if one can be found (otherwise None)
    """
    
    query_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{}/cids/txt?name_type=word"
    
    # If a timeout error is received, wait 30s, then try again.
    try:
        reply = requests.get(query_url.format(name), timeout = 30)
    except ConnectTimeout:
        sleep(30)
        reply = requests.get(query_url.format(name), timeout = 30)
    
    if reply.status_code == 200:
        return reply.text.split("\n")[0]
    
    # on a failed search, try again using a cleaned name
    else:
        reply = requests.get(query_url.format(clean_name(name)), timeout = 30)
        if reply.status_code == 200:
            return reply.text.split("\n")[0]
        else:
            return None

We'll also want the option to *aggressively* cleaning the chemical's name if & only if a match cannot be found on the unmodified name. Note that this may result in a lower fidelity to the original chemical information (for example, an enantiopure compound being reported as a racemate).

In [6]:
import re

def clean_name(dirty_name: str) -> str:
    """
    Called:
        By validate function.
    Purpose:
        Some molecule names contain strings not interpretable by the CACTUS
        API, such as "(+/-)" to indicate racemates. In their current form,
        a CAS number or structure cannot be found.
        This function aggressively removes such strings in an attempt to
        allow the API lookup, with the cost of decreased fidelity to the 
        structural information present in the original name.
    Accepts:
        dirty_name: a string that contains a molecule's name, currently
            uninterpretable by the CACTUS API.
    Returns:
        an aggressively cleaned name.
    """
    clean_name = dirty_name.lower()
    clean_name = re.sub(r"(cis)?(trans)?", "", clean_name)           # remove cis/trans isomerism
    clean_name = re.sub(r"\((\+)?(\/)?([\-\–])?\)", "", clean_name)  # remove optical rotations
    clean_name = re.sub(r"\sand\s", "", clean_name)                  # remove " and "
    clean_name = re.sub(r"\([\w\-\s\,]*\)", "", clean_name)          # remove all parentheticals.
    clean_name = re.sub(r"\[[\w\-\s\,]*\]", "", clean_name)          # remove everything in square brackets
    clean_name = re.sub(r"(\-)\1", "-", clean_name)                  # remove duplicative hyphens
    clean_name = re.sub(r"(?<=[0-9]).(?=[0-9])", ",", clean_name)    # separate numbers by commas, not hyphens
    clean_name = clean_name.strip("-")                               # remove leading/trailing hyphens
    clean_name = clean_name.strip()                                  # remove leading/trailing spaces
    
    return clean_name    

In [7]:
import json

def get_pubchem_data(pubchem_id: str) -> dict:
    """
    Called:
        By the `validate` function.
    Purpose:
        Find a key molecular data given the molecule's unique PubChem ID.
    Accepts:
        pubchem_id: the molecule's unique PubChem ID number
    Returns:
        a dict of chemical information about the molecule, with keys:
            "SMILES_code": chemical structure encoded as a SMILES string,
            "InChI_code": chemical structure encoded as an InChI string,
            "molecular_formula": the molecular formula,
            "molecular_weight_amu": the molecular weight in units of amu,
            "common_name": the molecule's common name.
    """
    
    query_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/property/CanonicalSMILES,InChI,MolecularFormula,MolecularWeight,Title/json"
    
    if pubchem_id:
        
        # If a timeout error is received, wait 30s, then try again.
        try:
            reply = requests.get(query_url.format(pubchem_id), timeout = 30)
        except ConnectTimeout:
            sleep(30)
            reply = requests.get(query_url.format(pubchem_id), timeout = 30)
    
        if reply.status_code == 200 and reply.json():
            return reply.json()["PropertyTable"]["Properties"][0]
    else:
        return None

Finally, a validation function to manage the functions above:

In [8]:
def manage_query(mol: dict) -> dict:
    """
    Called:
        By user.
    Purpose:
        Uses the PubChem API to retrieve additional chemical information by
        looking up a chemical name (or, if missing, a CAS number). Retrieved
        information is: chemical structure as both SMILES and InChI codes,
        molecular formula, molecular weight, and common name).
    Accepts:
        mol: a dict describing a single molecule (with keys name, CAS_num,
            aroma, and flavor).
    Returns:
        a dict with validated and expanded chemical information, with the keys:
            "SMILES_code": chemical structure encoded as a SMILES string,
            "InChI_code": chemical structure encoded as an InChI string,
            "molecular_formula": the molecular formula,
            "molecular_weight_amu": the molecular weight in units of amu,
            "common_name": the molecule's common name.
            "aroma": preserved smell data from the input dict.
            "flavor": preserved flavor data from the input dict.
        Note that neither the input name nor the input CAS number are preserved.
    """
    
    # Try to look up the PubChem ID number, first by the molecule's name, 
    # falling back to CAS number as a backup.
    # Missing values are present as an empty string, meaning we cannot simply
    # check for their existence as indicative of having a useful value. Check
    # by length instead.
    pubchem_id = None
    if len(mol["name"]) > 0:
        pubchem_id = get_pubchem_id(mol["name"])
                
    elif len(mol["CAS_num"]) > 5:
        pubchem_id = get_pubchem_id(mol["CAS_num"])

    # If a PubChem ID was found, retrieve additional data. Otherwise, return None.
    if pubchem_id:
        mol_output = get_pubchem_data(pubchem_id)
        mol_output["flavor"] = mol["flavor"]
        mol_output["aroma"] = mol["aroma"]
        # Rename keys to something more meaningful to me:
        mol_output["SMILES_code"] = mol_output.pop("CanonicalSMILES", None)
        mol_output["InChI_code"] = mol_output.pop("InChI", None)
        mol_output["molecular_formula"] = mol_output.pop("MolecularFormula", None)
        mol_output["molecular_weight_amu"] = mol_output.pop("MolecularWeight", None)
        mol_output["common_name"] = mol_output.pop("Title", None)
        mol_output.pop("CID", None)
    else:
        mol_output = dict(name = mol["name"],
                          status = "Not found in PubChem")

    return mol_output     

## Test the validation functions

With the validation functions in place, let's double-check that they are working as intended by testing them on just one of the molecules in the dataset.

But first, a hacky workaround (below). I've found that `requests` is taking an exhorbitantly long time because IPv6 is broken somewhere along the line. This workaround makes urllib default to IPv4, which dramatically sped up requests. Thanks, [Stack Overflow](https://stackoverflow.com/questions/62599036/python-requests-is-slow-and-takes-very-long-to-complete-http-or-https-request).

In [9]:
import socket
import requests.packages.urllib3.util.connection as urllib3_cn
 
def allowed_gai_family():
    return socket.AF_INET
 
urllib3_cn.allowed_gai_family = allowed_gai_family

Now we're ready to actually try the lookup functions:

In [10]:
manage_query(aromas_flavors[0])

{'flavor': '',
 'aroma': 'fruity floral fatty fruity|citrusy',
 'SMILES_code': 'CC1CCC2=C(C1)OC=C2C',
 'InChI_code': 'InChI=1S/C10H14O/c1-7-3-4-9-8(2)6-11-10(9)5-7/h6-7H,3-5H2,1-2H3/t7-/m1/s1',
 'molecular_formula': 'C10H14O',
 'molecular_weight_amu': '150.22',
 'common_name': '(+)-Menthofuran'}

Great! This is working as expected.

## Apply the validation functions to the full data set

Now that the validation functions are ready to go, it's time to turn them loose on the full data set. We'll use a progress bar from `tqdm` to give greater transparency about how it is going.

In [11]:
from tqdm import tqdm

In [12]:
aromas_flavors_validated = [manage_query(entry) for entry in tqdm(aromas_flavors)]

100%|██████████| 2457/2457 [46:16<00:00,  1.13s/it] 


All done. How does everything look?

In [ ]:
aromas_flavors_validated[:5]

[{'flavor': '',
  'aroma': 'fruity floral fatty fruity|citrusy',
  'SMILES_code': 'CC1CCC2=C(C1)OC=C2C',
  'InChI_code': 'InChI=1S/C10H14O/c1-7-3-4-9-8(2)6-11-10(9)5-7/h6-7H,3-5H2,1-2H3/t7-/m1/s1',
  'molecular_formula': 'C10H14O',
  'molecular_weight_amu': '150.22',
  'common_name': '(+)-Menthofuran'},
 {'flavor': 'vegetal fruity sharp|chemical liquorlike',
  'aroma': 'fatty dairy',
  'SMILES_code': 'CCCCC(C=C)O',
  'InChI_code': 'InChI=1S/C7H14O/c1-3-5-6-7(8)4-2/h4,7-8H,2-3,5-6H2,1H3',
  'molecular_formula': 'C7H14O',
  'molecular_weight_amu': '114.19',
  'common_name': '1-Hepten-3-OL'},
 {'flavor': 'fruity|berry fruity|citrusy',
  'aroma': 'fruity fruity|citrusy',
  'SMILES_code': 'CC(CCC=C(C)C)CC(C)(C)O',
  'InChI_code': 'InChI=1S/C12H24O/c1-10(2)7-6-8-11(3)9-12(4,5)13/h7,11,13H,6,8-9H2,1-5H3',
  'molecular_formula': 'C12H24O',
  'molecular_weight_amu': '184.32',
  'common_name': '2,4,8-Trimethyl-7-nonen-2-OL'},
 {'flavor': 'vegetal fruity sharp|sulfurous fruity|citrusy',
  'aroma'

Perfect. The data validation / additional lookup functions have standardized some of these names & provided additional chemical data.

## Save the expanded data set

The expanded data set will be exported to a new CSV for ease of sharing online. Before that, let's pull out only the entries that actually have chemical data. Recall from `manage_query()` that if a molecule from our input data set could not be found in the PubChem database, the resulting dict will contain the `status = "Not found in PubChem` key/value pair. This will be a convenient way to remove the un-matched molecules:

In [14]:
complete_mols = [mol for mol in aromas_flavors_validated if "status" not in mol.keys()]
print("Found PubChem data for this many molecules: ", len(complete_mols))

Found PubChem data for this many molecules:  1886


That's not too bad. The input data set had about 2400 entries, many of which were not even discrete chemicals (like "fennel" or "myrtle leaves"). It's reasonable that a fair number of entries were dropped here.

Write the CSV to wrap up:

In [16]:
with open("processed-data/aromas-flavors-validated.csv", "w") as file:
    filewriter = csv.DictWriter(file,
                                delimiter = "\t",
                                fieldnames = ["common_name", "SMILES_code", "InChI_code", "aroma", "flavor"],
                                extrasaction = "ignore")
    filewriter.writeheader()
    for molecule in aromas_flavors_validated:
        filewriter.writerow(molecule)